In [11]:
import networkx as nx
import numpy as np
import functools
import random

In [12]:
from optiLib.problem import NetGraph
from optiLib.problem import TaskGraph

def generate_adjacency_matrix(rows, cols):
    num_nodes = rows * cols
    adjacency_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

    for row in range(rows):
        for col in range(cols):
            node_index = row * cols + col
            if col > 0:  # узел слева
                adjacency_matrix[node_index][node_index - 1] = 1
            if col < cols - 1:  # узел справа
                adjacency_matrix[node_index][node_index + 1] = 1
            if row > 0:  # узел сверху
                adjacency_matrix[node_index][node_index - cols] = 1
            if row < rows - 1:  # узел снизу
                adjacency_matrix[node_index][node_index + cols] = 1

    return adjacency_matrix


martx = [[0, 1500, 1500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 500, 500, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 100, 0, 0, 250, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 250, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 250, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 100, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 350, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 350, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 400, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 500],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],]

net  = NetGraph(generate_adjacency_matrix(7, 7), net_power=(1500, 2500), e0=(10, 50), emax=(70,100))
task = TaskGraph(martx, w=(100,600))

In [13]:
random.choice([[1,2],2,3])

[1, 2]

In [14]:
import functools
import networkx as nx
import numpy as np

@functools.lru_cache
def create_paths(vector, task_edges, network_graph):
    paths = {}
    for start, end in task_edges:
        if vector[start] == vector[end]:
            paths[str(start) + str(end)] = [vector[start]]
        else:
            paths[str(start) + str(end)] = random.choice(list(nx.all_shortest_paths(network_graph, vector[start], vector[end])))
    return paths

@functools.lru_cache
def network_status_calculation(vector, problem):
    T_lim = problem.t_lim
    vector_tuple = tuple(vector) 
    task_edges = tuple((start, end) for start, end, _ in nx.to_edgelist(problem.task_graph.graph))
    network_graph = problem.network_graph.graph
    
    paths = create_paths(vector_tuple, task_edges, network_graph)
    
    net_number = network_graph.number_of_nodes()
    task_number = problem.task_graph.graph.number_of_nodes()

    W = [0] * net_number
    v_task_to_node = [0] * net_number
    v_sent_to_node = [0] * net_number
    v_reseive_to_node = [0] * net_number
    
    for start, end, weight in nx.to_edgelist(problem.task_graph.graph):
        path_key = str(start) + str(end)
        path = paths[path_key]
        
        if len(path) > 1:
            W[path[0]] += problem.task_graph.operations[start].w + weight['weight']
            v_sent_to_node[path[0]] += weight['weight']
            v_task_to_node[path[0]] += problem.task_graph.operations[start].w
            
            W[path[-1]] += problem.task_graph.operations[end].w + weight['weight']
            v_reseive_to_node[path[-1]] += weight['weight']
            v_task_to_node[path[-1]] += problem.task_graph.operations[end].w
            
            for node in path[1:-1]:
                W[node] += 2 * weight['weight']
                v_sent_to_node[node] += weight['weight']
                v_reseive_to_node[node] += weight['weight']
        else:
            W[vector[start]] += problem.task_graph.operations[start].w
            v_task_to_node[vector[start]] += problem.task_graph.operations[start].w
            
            W[vector[end]] += problem.task_graph.operations[end].w
            v_task_to_node[vector[end]] += problem.task_graph.operations[end].w
    
    vector_tuple = tuple(vector) 
    
    arr = sorted(nx.all_simple_paths(problem.task_graph.graph, 0, task_number - 1), key=len, reverse=True)
    
    T_task = [0] * task_number
    len_first_path = len(arr[0])
    time_per_task = T_lim / len_first_path
    
    for task in arr[0]:
        T_task[task] = time_per_task
    
    for path_task in arr[1:]:
        sum_t = sum(T_task[task] for task in path_task)
        temp = [task for task in path_task if T_task[task] == 0]
        
        if temp:  # если нашлись задачи без оценки времени
            time_remaining = max(0, T_lim - sum_t)
            time_per_task_temp = time_remaining / len(temp)
            for task in temp:
                T_task[task] = time_per_task_temp
    
    T_NET = [0] * net_number

    for start, end, _ in nx.to_edgelist(problem.task_graph.graph):
        path_net = paths[str(start) + str(end)]
        if len(path_net) == 1:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] or time_per_task, T_task[start], T_task[end])
        else:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] or T_task[end], T_task[end])
            time_start_share = T_task[start] / (len(path_net) - 1)
            for node in path_net[:-1]:
                T_NET[node] = min(T_NET[node] or time_start_share, time_start_share)
    
    special_t = T_lim / len_first_path
    result = [[] for _ in range(net_number)]
    for i, val in enumerate(vector):
        result[val].append(i)
    
    for node in range(net_number):
        common_tasks = set(result[node]) & set(arr[0])
        if len(common_tasks) > 1:
            T_NET[node] = special_t * len(common_tasks)
    
    D = [W[i] / (problem.network_graph.nodes[i].p * T_NET[i]) if T_NET[i] != 0 else 0 for i in range(net_number)]
    return {"w":W, "v_task_to_node":v_task_to_node, "v_sent_to_node":v_sent_to_node, "v_reseive_to_node":v_reseive_to_node, "D":D, "T_NET":T_NET, "T_task":T_task}

def c1(vector, problem):
    net_ststus = network_status_calculation(tuple(vector), problem)
    D = net_ststus['D']
    flag = all(d <= 1 for d in D) 
    return int(flag)

def f1(vector, problem):
    """
    Функция которая на основе полученого времени работы и загружености расичтывает энергопотребление
    """
    net_ststus = network_status_calculation(tuple(vector), problem)
    D = net_ststus['D']
    T_NET = net_ststus['T_NET']

    T_lim = problem.t_lim
    net_number = problem.network_graph.graph.number_of_nodes()

    E = [0 for _ in range(net_number)]
    Z = [0 for _ in range(net_number)]
    for i in range(net_number):
        E[i] = problem.network_graph.nodes[i].e0 + (problem.network_graph.nodes[i].eMax - problem.network_graph.nodes[i].e0) * D[i]
        Z[i] = E[i]*T_NET[i] + (T_lim - T_NET[i])* problem.network_graph.nodes[i].e0
    Z = sum(Z)
    return Z

def f2(vector, problem):
    """
    Вычисляет количество пересылок данных, необходимых при распределении задач
    по узлам сети, если изначально все задачи были на узле 0.
    
    Args:
        vector (list): Вектор распределения задач на узлы сети.
        problem (): network_graph (nx.Graph): Граф сети.
        
    Returns:
        int: Общее количество пересылок данных.
    """
    total_transfers = 0
    network_graph = problem.network_graph.graph

    for task_id, node_id in enumerate(vector):
        if node_id != 0:
            # Найти кратчайший путь от узла 0 до текущего узла
            path = nx.shortest_path(network_graph, source=0, target=node_id)
            
            # Увеличить счетчик пересылок на количество узлов в пути
            total_transfers += len(path) - 1
    
    return total_transfers + 1


In [15]:
import functools
import networkx as nx
import numpy as np
from joblib import Parallel, delayed

@functools.lru_cache
def create_paths(vector, task_edges, network_graph):
    # переходим на использование np для vector[start] и vector[end]
    paths = {}
    for start, end in task_edges:
        if vector[start] == vector[end]:
            paths[str(start) + str(end)] = [vector[start]]
        else:
            paths[str(start) + str(end)] = random.choice(list(nx.all_shortest_paths(network_graph, vector[start], vector[end])))
    return paths

@functools.lru_cache
def network_status_calculation(vector, problem):
    T_lim = problem.t_lim
    vector_tuple = tuple(vector) 
    task_edges = np.array([(start, end) for start, end, _ in nx.to_edgelist(problem.task_graph.graph)]) # используем np.array для оптимизации
    network_graph = problem.network_graph.graph
    
    # использование joblib для параллельного создания путей
    paths = create_paths(vector_tuple, task_edges, network_graph)
    
    net_number = network_graph.number_of_nodes()
    task_number = problem.task_graph.graph.number_of_nodes()

    W = np.zeros(net_number)
    v_task_to_node = np.zeros(net_number)
    v_sent_to_node = np.zeros(net_number)
    v_reseive_to_node = np.zeros(net_number)
    
    for start, end, weight in nx.to_edgelist(problem.task_graph.graph):
        path_key = str(start) + str(end)
        path = paths[path_key]
        
        if len(path) > 1:
            w_start = problem.task_graph.operations[start].w
            w_end = problem.task_graph.operations[end].w
            weight_value = weight['weight']
            
            W[path[0]] += w_start + weight_value
            v_sent_to_node[path[0]] += weight_value
            v_task_to_node[path[0]] += w_start
            
            W[path[-1]] += w_end + weight_value
            v_reseive_to_node[path[-1]] += weight_value
            v_task_to_node[path[-1]] += w_end
            
            W[path[1:-1]] += 2 * weight_value
            v_sent_to_node[path[1:-1]] += weight_value
            v_reseive_to_node[path[1:-1]] += weight_value
        else:
            w_start = problem.task_graph.operations[start].w
            w_end = problem.task_graph.operations[end].w
            W[vector[start]] += w_start
            v_task_to_node[vector[start]] += w_start
            
            W[vector[end]] += w_end
            v_task_to_node[vector[end]] += w_end
    
    arr = sorted(nx.all_simple_paths(problem.task_graph.graph, 0, task_number - 1), key=len, reverse=True)
    len_first_path = len(arr[0])
    time_per_task = T_lim / len_first_path
    
    T_task = np.zeros(task_number)
    T_task[arr[0]] = time_per_task
    
    for path_task in arr[1:]:
        sum_t = T_task[path_task].sum()
        temp = np.array([task for task in path_task if T_task[task] == 0])
        
        if temp.size:  # если нашлись задачи без оценки времени
            time_remaining = max(0, T_lim - sum_t)
            time_per_task_temp = time_remaining / len(temp)
            T_task[temp] = time_per_task_temp

    T_NET = np.zeros(net_number)

    for start, end, _ in nx.to_edgelist(problem.task_graph.graph):
        path_net = paths[str(start) + str(end)]
        if len(path_net) == 1:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] if T_NET[path_net[-1]] else time_per_task, T_task[start], T_task[end])
        else:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] if T_NET[path_net[-1]] else T_task[end], T_task[end])
            time_start_share = T_task[start] / (len(path_net) - 1)
            for node in path_net[:-1]:
                T_NET[node] = min(T_NET[node] if T_NET[node] else time_start_share, time_start_share)

    special_t = T_lim / len_first_path
    result = [[] for _ in range(net_number)]
    for i, val in enumerate(vector):
        result[val].append(i)

    for node in range(net_number):
        common_tasks = set(result[node]) & set(arr[0])
        if len(common_tasks) > 1:
            T_NET[node] = special_t * len(common_tasks)

    D = np.array([W[i] / (problem.network_graph.nodes[i].p * T_NET[i]) if T_NET[i] != 0 else 0 for i in range(net_number)])
    return {"w": W, "v_task_to_node": v_task_to_node, "v_sent_to_node": v_sent_to_node, "v_reseive_to_node": v_reseive_to_node, "D": D, "T_NET": T_NET, "T_task": T_task}

def c1(vector, problem):
    net_status = network_status_calculation(vector, problem)
    D = net_status['D']
    flag = all(d <= 1 for d in D)
    return int(flag)

def f1(vector, problem):
    """
    Функция, которая на основе полученного времени работы и загруженности рассчитывает энергопотребление.
    """
    net_status = network_status_calculation(vector, problem)
    D = net_status['D']
    T_NET = net_status['T_NET']

    T_lim = problem.t_lim
    net_number = problem.network_graph.graph.number_of_nodes()

    E = np.array([problem.network_graph.nodes[i].e0 + (problem.network_graph.nodes[i].eMax - problem.network_graph.nodes[i].e0) * D[i] for i in range(net_number)])
    Z = np.sum(E * T_NET + (T_lim - T_NET) * np.array([problem.network_graph.nodes[i].e0 for i in range(net_number)]))
    return Z

def f2(vector, problem):
    """
    Вычисляет количество пересылок данных, необходимых при распределении задач
    по узлам сети, если изначально все задачи были на узле 0.

    Args:
        vector (list): Вектор распределения задач на узлы сети.
        problem (): network_graph (nx.Graph): Граф сети.

    Returns:
        int: Общее количество пересылок данных.
    """
    total_transfers = 0
    network_graph = problem.network_graph.graph

    for task_id, node_id in enumerate(vector):
        if node_id != 0:
            # Найти кратчайший путь от узла 0 до текущего узла
            path = nx.shortest_path(network_graph, source=0, target=node_id)

            # Увеличить счетчик пересылок на количество узлов в пути
            total_transfers += len(path) - 1

    return total_transfers + 1
  


In [16]:
from optiLib import NetworkOptimizationProblem

problem = NetworkOptimizationProblem(net, task, f_objective=[f1,f2], f_constraints=[c1])

In [17]:
from optiLib.optimizers import RandomSearchOptimizer, ParticleSwarmOptimizer

# Инициализация оптимизатора
optimizer = ParticleSwarmOptimizer(problem, iterations=1000)

# Запуск оптимизации
best_solution, best_value = optimizer.optimize()
from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

Optimizing:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
from optiLib.optimizers import RandomSearchOptimizer

# Инициализация оптимизатора
optimizer = RandomSearchOptimizer(problem, num_iterations= 3000)

# Запуск оптимизации
best_solution, best_value = optimizer.optimize()
from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

Optimizing:   0%|          | 0/3000 [00:00<?, ?it/s]

Optimizing: 100%|██████████| 3000/3000 [00:18<00:00, 164.49it/s]

Лучшее решение: [ 2 13  9 24 37 28 37 37 18 29 10 36 37  0  8 15]
Значение целевой функции: 21383836.05974204


In [ ]:
from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

Лучшее решение: [ 2 13  9 24 37 28 37 37 18 29 10 36 37  0  8 15]
Значение целевой функции: 21383836.05974204


In [ ]:
print(np.log(problem.evaluate(np.array([ 0,21, 0, 21, 21,  8,  0, 28, 21, 35,  0, 28, 21, 21, 14, 21]))))

16.19308760277134


In [ ]:
print(np.log(problem.evaluate(np.array([ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))))

12.52618081792831
